This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '<your_api_key>'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
from requests import get
from json import loads

db_code = 'FSE'
ticker = 'AFX_X'
url = 'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.json?api_key={key}'
url = url.format(database_code=db_code,dataset_code=ticker,key=API_KEY)
drange = '&start_date={start_date}&end_date={end_date}'.format(start_date='2017-01-01',end_date='2018-01-01')
url = url+drange
print(url)

https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=oJWR6X2ssaig65tkSPbw&start_date=2017-01-01&end_date=2018-01-01


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

quandl_json = get(url).json()
print(quandl_json['dataset_data']['data'][0])
#print(quandl_json.json())

['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [9]:
for key, value in quandl_json.items():
    print("First Key: ",key)
    print(type(value))
    #display(value)
    #break
    for k,v in value.items():
        print(k)
        #display(v)
        if k == 'data' or v == 'data':
            break

print("Column Names: ",quandl_json['dataset_data']['column_names'])

First Key:  dataset_data
<class 'dict'>
limit
transform
column_index
column_names
start_date
end_date
frequency
data
Column Names:  ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
#1. & 2.
db_code = 'FSE'
ticker = 'AFX_X'
url = 'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.json?api_key={key}'
url = url.format(database_code=db_code,dataset_code=ticker,key=API_KEY)
drange = '&start_date={start_date}&end_date={end_date}'.format(start_date='2017-01-01',end_date='2018-01-01')
url = url+drange
print(url)

quandl_json = get(url).json()

https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=oJWR6X2ssaig65tkSPbw&start_date=2017-01-01&end_date=2018-01-01


In [21]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
openings = {}
for i in quandl_json['dataset_data']['data']:
    if i[1] and i[0]:
        openings[i[0]] = i[1]

sorted_openings = sorted(openings.items(), key=lambda kv: kv[1])

high_open = sorted_openings[-1]
low_open = sorted_openings[0]

In [25]:
#4. What was the largest change in any one day (based on High and Low price)?
changes = {}
for i in quandl_json['dataset_data']['data']:
    if i[3] and i[2] and i[0]:
        changes[i[0]] = i[2] - i[3]

sorted_ch = sorted(changes.items(), key=lambda kv: kv[1])
large_ch = sorted_ch[-1]

In [33]:
#5. What was the largest change between any two days (based on Closing Price)?
q_data = quandl_json['dataset_data']['data']
changesC = {}

for i in range(len(q_data)):
    try:
        changesC[q_data[i][0]] = q_data[i+1][4] - q_data[i][4]
    except IndexError:
        break
    
sorted_chC = sorted(changesC.items(), key=lambda kv: kv[1])
large_chC = sorted_chC[-1]

In [37]:
#6. What was the average daily trading volume during this year? Trade Volume = 6 element
vol_sum = 0
vol_med = []
for i in q_data:
    vol_sum = vol_sum + i[6]
    vol_med.append(i[6])

vol_av = vol_sum/len(q_data)

In [46]:
#7. (Optional) What was the median trading volume during this year. 
#(Note: you may need to implement your own function for calculating the median.)
vol_med = sorted(vol_med)
medi = vol_med[round(len(vol_med)/2)-1]
print(medi)

76286.0
